In [1]:
using Oceananigans 
using Oceananigans.Units
using Oceananigans.OutputReaders: FieldTimeSeries
using Oceananigans.BoundaryConditions
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
using CUDA: CuArray
using CairoMakie 
using NCDatasets

In [2]:
# Model parameters
const Nx = 300
const Nz = 80
const f = 1e-4               # Coriolis frequency [s⁻¹]
const L_front = 10kilometers  # Initial front width [m]
const aspect_ratio = 100      # L_front/H
const δ = 0                   # Strain ratio (α/f)
const Ro = 0.1             # Rossby number (defines M^2)
const F = Inf              # Froude number (N² = M²/(F²H))
const Re_h = 1e10     # horizontal reynolds number
const Re_v = +Inf              #vertical reynolds number
const n = 2              #diffusivity number     

# Derived parameters
H_front = L_front/aspect_ratio
α = f*δ
M² = (Ro^2*f^2*L_front)/H_front
N² = (M²*L_front)/(F^2*H_front)
Bu = Ro/F
Δb = M²*L_front
κh = (sqrt(Δb*H_front)*L_front^(n-1))/Re_h
κv = κh*(Re_h/Re_v)*(H_front/L_front)^n

#filename = "δ="*string(δ)*"_Ro="*string(Ro)*"_F="*string(F)*"_Re_h="*string(Re_h)
filename = "periodic_adjustment"

println("Filename: ", filename)
println("\nDerived parameters:")
println("H = ", H_front/1000, " km")
println("α = ", α, " s⁻¹")
println("M² = ", M², " s⁻²")
println("N² = ", N², " s⁻²")
println("Bu = ", Bu)
println("κh = ", κh)
println("κv = ", κv)

Filename: periodic_adjustment

Derived parameters:
H = 0.1 km
α = 0.0 s⁻¹
M² = 1.0000000000000002e-8 s⁻²
N² = 0.0 s⁻²
Bu = 0.0
κh = 1.0e-7
κv = 0.0


In [3]:

grid = RectilinearGrid(GPU(),size = (Nx, Nz), #in ST15 they use Nx = 200*L_front, Nz = 100*H_front
                       x = (-L_front/2, L_front/2),
                       z = (-H_front, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

In [4]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-7, κ=1.0e-7), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0, κ=0.0))

In [5]:
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = (:b,:T),
                advection = WENO(),
                closure = closure)

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: (b, T)
├── closure: Tuple with 2 closures:
│   ├── HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-7, κ=(b=1.0e-7, T=1.0e-7))
│   └── VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0, κ=(b=0.0, T=0.0))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [6]:
bᵢ(x, z) = Δb*sin(2*pi/L_front * x)
Tᵢ(x, z) = exp(-(x/(L_front/20)).^2)

set!(model, b= bᵢ, T= Tᵢ, u = 0, v = 0, w = 0)  # Start from rest

In [7]:
simulation = Simulation(model, Δt=20minutes, stop_time=5days)


Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 5 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [8]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [9]:

using Printf

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(50))

In [10]:
# Output setup
g = 9.81
u, v, w = model.velocities
ζ = ∂z(u) - ∂x(w)  # Vorticity in x-z plane
b = model.tracers.b
wc = Field(@at (Center, Center, Center) model.velocities.w)
T = model.tracers.T

#For Lagrangian filtering
simulation.output_writers[:jld2fields] = JLD2Writer(
    model, (; b, u, v, w, wc,T), filename=filename * ".jld2", schedule=TimeInterval(1hour), overwrite_existing=true)


#for python visualisation
rm(filename * ".nc",force=true)
simulation.output_writers[:ncfields] = NetCDFWriter(
    model, (; b, u, v, w, wc,T), filename=filename * ".nc", schedule=TimeInterval(1hour), overwrite_existing=true)
     

NetCDFWriter scheduled on TimeInterval(1 hour):
├── filepath: periodic_adjustment.nc
├── dimensions: time(0), x_faa(300), x_caa(300), z_aaf(81), z_aac(80)
├── 6 outputs: (v, w, T, wc, b, u)
├── array_type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 27.5 KiB

In [11]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 26.582 seconds, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (15.687 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (2.360 seconds).


[09.40%] i: 50, t: 11.275 hours, wall time: 4.880 seconds, max(u): (2.399e-02, 5.051e-02, 4.262e-04) m/s, next Δt: 8.262 minutes
[12.55%] i: 100, t: 15.062 hours, wall time: 214.734 ms, max(u): (2.436e-02, 1.238e-02, 3.869e-04) m/s, next Δt: 4.097 minutes
[15.55%] i: 150, t: 18.661 hours, wall time: 198.345 ms, max(u): (1.268e-02, 2.712e-03, 2.549e-04) m/s, next Δt: 4.957 minutes
[18.59%] i: 200, t: 22.314 hours, wall time: 218.476 ms, max(u): (3.035e-02, 3.555e-02, 5.420e-04) m/s, next Δt: 3.660 minutes
[21.26%] i: 250, t: 1.063 days, wall time: 201.139 ms, max(u): (9.999e-03, 6.040e-02, 2.134e-04) m/s, next Δt: 4.428 minutes
[24.54%] i: 300, t: 1.227 days, wall time: 217.224 ms, max(u): (2.775e-02, 4.511e-02, 5.895e-04) m/s, next Δt: 4.002 minutes
[27.11%] i: 350, t: 1.355 days, wall time: 200.701 ms, max(u): (2.499e-02, 1.353e-02, 4.506e-04) m/s, next Δt: 3.950 minutes
[30.07%] i: 400, t: 1.503 days, wall time: 218.103 ms, max(u): (1.282e-02, 3.512e-03, 3.421e-04) m/s, next Δt: 5.25

[ Info: Simulation is stopping after running for 26.311 seconds.
[ Info: Simulation time 5 days equals or exceeds stop time 5 days.
[ Info: Simulation completed in 26.350 seconds


In [12]:
using JLD2
file = jldopen(filename * ".jld2")
print(keys(file["timeseries/u"]))
close(file)

["serialized", "0", "3", "6", "9", "12", "15", "18", "22", "28", "34", "40", "48", "56", "67", "82", "99", "114", "128", "142", "155", "167", "179", "195", "212", "228", "243", "257", "270", "283", "295", "309", "325", "342", "358", "372", "386", "399", "411", "423", "437", "453", "469", "485", "500", "513", "526", "538", "551", "565", "582", "598", "614", "628", "643", "655", "667", "678", "693", "710", "727", "742", "756", "769", "782", "794", "808", "824", "841", "856", "870", "884", "897", "909", "921", "936", "952", "968", "984", "999", "1013", "1026", "1039", "1056", "1077", "1099", "1120", "1139", "1156", "1173", "1188", "1203", "1221", "1241", "1262", "1283", "1304", "1323", "1340", "1356", "1376", "1401", "1429", "1455", "1479", "1500", "1519", "1537", "1557", "1580", "1605", "1631", "1658", "1687", "1714", "1738", "1759", "1778", "1805", "1840", "1878", "1913"]